#### Task 1

Set up cssandra cluster of three nodes
* https://hub.docker.com/_/cassandra 
* https://gokhanatil.com/2018/02/build-a-cassandra-cluster-on-docker.html 
* https://www.jamescoyle.net/how-to/2448-create-a-simple-cassandra-cluster-with-3-nodes
* https://www.digitalocean.com/community/tutorials/how-to-run-a-multi-node-cluster-database-with-cassandra-on-ubuntu-14-04


Cassandra cluster is configured with docker compose.

Check file *docker-compose.yml* in this repo.

    >docker ps
    CONTAINER ID   IMAGE            COMMAND                  CREATED              STATUS                             PORTS
                          NAMES
    0a2eb4678003   cassandra:3.11   "docker-entrypoint.s…"   About a minute ago   Up 43 seconds (health: starting)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9044->9042/tcp   node_3
    2d9c07c5d599   cassandra:3.11   "docker-entrypoint.s…"   About a minute ago   Up About a minute (healthy)        7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9042->9042/tcp   node_1
    4c791911e5cd   cassandra:3.11   "docker-entrypoint.s…"   About a minute ago   Up About a minute (healthy)        7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9043->9042/tcp   node_2

#### Task 2

Check the cluster configuration with *nodetool status*

    >docker exec node_1 nodetool status
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  73.7 KiB   128          65.8%             959bed54-9aa4-413c-ad06-3dd6bedde88e  west
    UN  172.19.0.2  73.82 KiB  128          70.8%             e5a87163-e235-468b-9559-d359632e945e  west
    UN  172.19.0.4  87.67 KiB  128          63.4%             b765664c-de5e-4b8a-af56-a34b718c2225  west


#### Task 3

Using *cqlsh* create three Keyspaces
* https://www.tutorialspoint.com/cassandra/cassandra_create_keyspace.htm
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/create_keyspace_r.html


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> DESC keyspaces;

    system_traces  system_schema  system_auth  system  system_distributed

    cqlsh> CREATE KEYSPACE music_db_r1 WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '1'};
    cqlsh> CREATE KEYSPACE music_db_r2 WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '2'};
    cqlsh> CREATE KEYSPACE music_db_r3 WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '3'};
    cqlsh> DESC keyspaces;

    system_schema  system       music_db_r2  system_distributed
    system_auth    music_db_r1  music_db_r3  system_traces


#### Task 4

Create tables in every Keyspace
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/create_table_r.html
* https://www.tutorialspoint.com/cassandra/cassandra_create_table.htm


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> CREATE TABLE music_by_genre (genre VARCHAR, performer VARCHAR, year INT, title VARCHAR, comments VARCHAR, PRIMARY KEY ((genre), performer, year, title)) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC);
    cqlsh:music_db_r1> DESC music_by_genre;

    CREATE TABLE music_db_r1.music_by_genre (
        genre text,
        performer text,
        year int,
        title text,
        comments text,
        PRIMARY KEY (genre, performer, year, title)
    ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}    
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> CREATE TABLE music_by_genre (genre VARCHAR, performer VARCHAR, year INT, title VARCHAR, comments VARCHAR, PRIMARY KEY ((genre), performer, year, title)) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC);
    cqlsh:music_db_r2> DESC music_by_genre;

    CREATE TABLE music_db_r2.music_by_genre (
        genre text,
        performer text,
        year int,
        title text,
        comments text,
        PRIMARY KEY (genre, performer, year, title)
    ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}    
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> CREATE TABLE music_by_genre (genre VARCHAR, performer VARCHAR, year INT, title VARCHAR, comments VARCHAR, PRIMARY KEY ((genre), performer, year, title)) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC);
    cqlsh:music_db_r3> DESC music_by_genre;

    CREATE TABLE music_db_r3.music_by_genre (
        genre text,
        performer text,
        year int,
        title text,
        comments text,
        PRIMARY KEY (genre, performer, year, title)
    ) WITH CLUSTERING ORDER BY (performer ASC, year DESC, title ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}    
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';


#### Task 5

Try to read and write to/from different nodes

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1991, 'Smells Like Teen Spirit');        
    cqlsh:music_db_r1> SELECT * FROM music_by_genre;

    genre | performer | year | title                   | comments
    -------+-----------+------+-------------------------+----------
    Rock |   Nirvana | 1991 | Smells Like Teen Spirit |     null

    (1 rows)

    >docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> SELECT * FROM music_by_genre;

    genre | performer | year | title                   | comments
    -------+-----------+------+-------------------------+----------
    Rock |   Nirvana | 1991 | Smells Like Teen Spirit |     null

    (1 rows)

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> SELECT * FROM music_by_genre;

    genre | performer | year | title                   | comments
    -------+-----------+------+-------------------------+----------
    Rock |   Nirvana | 1991 | Smells Like Teen Spirit |     null

    (1 rows)

    >docker exec -it node_2 cqlsh  
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Rock', 'The Beatles', 1963, 'Please Please Me', '+');
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title            | comments
    -------+-------------+------+------------------+----------
      Rock | The Beatles | 1963 | Please Please Me |        +

    (1 rows)

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title            | comments
    -------+-------------+------+------------------+----------
      Rock | The Beatles | 1963 | Please Please Me |        +

    (1 rows)

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Rock', 'The Beatles', 1965, 'Help!', '+');       
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title            | comments
    -------+-------------+------+------------------+----------
      Rock | The Beatles | 1965 |            Help! |        +
      Rock | The Beatles | 1963 | Please Please Me |        +

    (2 rows)

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> SELECT * FROM music_by_genre;

    genre | performer   | year | title            | comments
    -------+-------------+------+------------------+----------
      Rock | The Beatles | 1965 |            Help! |        +
      Rock | The Beatles | 1963 | Please Please Me |        +

    (2 rows)



    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1975, 'Dark Lady', '++');
    InvalidRequest: Error from server: code=2200 [Invalid query] message="Unmatched column names/values"
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Rock', 'Scorpions', 1975, 'Dark Lady', '++');    
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;

    genre | performer | year | title     | comments
    -------+-----------+------+-----------+----------
    Rock | Scorpions | 1975 | Dark Lady |       ++

    (1 rows)

    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;

    genre | performer | year | title     | comments
    -------+-----------+------+-----------+----------
    Rock | Scorpions | 1975 | Dark Lady |       ++

    (1 rows)
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1979, 'Another Piece of Meat');        
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Rock', 'Scorpions', 1975, 'Dark Lady', '+');     
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;

    genre | performer | year | title                 | comments
    -------+-----------+------+-----------------------+----------
    Rock | Scorpions | 1979 | Another Piece of Meat |     null
    Rock | Scorpions | 1975 |             Dark Lady |        +

    (2 rows)

    >docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;

    genre | performer | year | title                 | comments
    -------+-----------+------+-----------------------+----------
    Rock | Scorpions | 1979 | Another Piece of Meat |     null
    Rock | Scorpions | 1975 |             Dark Lady |        +

    (2 rows)

#### Task 6

Insert data to tables and check their distribution between nodes (for each Keyspace use *nodetool status*)
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/insert_r.html
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/select_r.html 
* https://www.tutorialspoint.com/cassandra/cassandra_create_data.htm
* https://www.tutorialspoint.com/cassandra/cassandra_read_data.htm


        >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1991, 'Drain You');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1989, 'Blew');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1992, 'Mexican Seafood');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Nirvana', 1993, 'Pennyroyal Tea');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Otis Rush', 1958, 'All Your Love (I Miss Loving)');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Muddy Waters', 1956, 'Got My Mojo Working');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Country', 'Johnny Duncan', 1976, 'Thinking of a Rendezvous');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Grandmaster Flash and the Furious Five', 1982, 'The Message');
    cqlsh:music_db_r1> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Public Enemy', 1989, 'Fight the Power');

    >docker exec -it node_1 nodetool status music_db_r1
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  74.5 KiB   128          33.6%             959bed54-9aa4-413c-ad06-3dd6bedde88e  west
    UN  172.19.0.2  74.62 KiB  128          34.3%             e5a87163-e235-468b-9559-d359632e945e  west
    UN  172.19.0.4  135.13 KiB  128          32.0%             b765664c-de5e-4b8a-af56-a34b718c2225  west



    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1965, 'Bad Boy');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1969, 'Come Together');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1970, 'Dig It');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'The Beatles', 1966, 'Yellow Submarine');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Otis Rush', 1958, 'All Your Love (I Miss Loving)');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Muddy Waters', 1956, 'Got My Mojo Working');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Country', 'Johnny Duncan', 1976, 'Thinking of a Rendezvous');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Grandmaster Flash and the Furious Five', 1982, 'The Message');
    cqlsh:music_db_r2> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Public Enemy', 1989, 'Fight the Power');

    >docker exec -it node_1 nodetool status music_db_r2 
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  74.5 KiB   128          65.8%             959bed54-9aa4-413c-ad06-3dd6bedde88e  west
    UN  172.19.0.2  74.62 KiB  128          70.8%             e5a87163-e235-468b-9559-d359632e945e  west
    UN  172.19.0.4  135.13 KiB  128          63.4%             b765664c-de5e-4b8a-af56-a34b718c2225  west


    >docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1990, 'Wind Of Change');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1980, 'The Zoo');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 2010, 'Raised on Rock');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Rock', 'Scorpions', 1980, 'Lady Starlight');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Otis Rush', 1958, 'All Your Love (I Miss Loving)');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Blues', 'Muddy Waters', 1956, 'Got My Mojo Working');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Country', 'Johnny Duncan', 1976, 'Thinking of a Rendezvous');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Grandmaster Flash and the Furious Five', 1982, 'The Message');
    cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title) VALUES ('Hip-Hop', 'Public Enemy', 1989, 'Fight the Power');

    >docker exec -it node_1 nodetool status music_db_r3
    Datacenter: dc_1
    ================
    Status=Up/Down
    |/ State=Normal/Leaving/Joining/Moving
    --  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
    UN  172.19.0.3  74.5 KiB   128          100.0%            959bed54-9aa4-413c-ad06-3dd6bedde88e  west
    UN  172.19.0.2  74.62 KiB  128          100.0%            e5a87163-e235-468b-9559-d359632e945e  west
    UN  172.19.0.4  135.13 KiB  128          100.0%            b765664c-de5e-4b8a-af56-a34b718c2225  west


#### Task 7

For each record from each keyspace show nodes where they are stored
* https://docs.datastax.com/en/dse/5.1/dse-admin/datastax_enterprise/tools/nodetool/toolsGetEndPoints.html

Continue here!!!!!!!

    >docker exec -it node_1 nodetool getendpoints music_db_r1 music_by_genre rock
    172.19.0.2

    >docker exec -it node_1 nodetool getendpoints music_db_r1 music_by_genre blues
    172.19.0.2

    >docker exec -it node_1 nodetool getendpoints music_db_r1 music_by_genre country
    172.19.0.3

    >docker exec -it node_1 nodetool getendpoints music_db_r1 music_by_genre hip-hop
    172.19.0.2

    >docker exec -it node_1 nodetool getendpoints music_db_r2 music_by_genre rock    
    172.19.0.2
    172.19.0.3

    >docker exec -it node_1 nodetool getendpoints music_db_r2 music_by_genre blues   
    172.19.0.2
    172.19.0.3

    >docker exec -it node_1 nodetool getendpoints music_db_r2 music_by_genre country 
    172.19.0.3
    172.19.0.2

    >docker exec -it node_1 nodetool getendpoints music_db_r2 music_by_genre hip-hop 
    172.19.0.2
    172.19.0.3

    >docker exec -it node_1 nodetool getendpoints music_db_r3 music_by_genre rock    
    172.19.0.2
    172.19.0.3
    172.19.0.4

    >docker exec -it node_1 nodetool getendpoints music_db_r3 music_by_genre blues   
    172.19.0.2
    172.19.0.3
    172.19.0.4

    >docker exec -it node_1 nodetool getendpoints music_db_r3 music_by_genre country 
    172.19.0.3
    172.19.0.2
    172.19.0.4

    >docker exec -it node_1 nodetool getendpoints music_db_r3 music_by_genre hip-hop 
    172.19.0.2
    172.19.0.3
    172.19.0.4

#### Task 8

Switch of one of the nodes. For each keyspace figure out a consistency level for reading and writing, which of them guaranty strong consistency
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/consistency_r.html


    >docker ps
    CONTAINER ID   IMAGE            COMMAND                  CREATED         STATUS                   PORTS
           NAMES
    a94cdfddcd55   cassandra:3.11   "docker-entrypoint.s…"   7 minutes ago   Up 7 minutes (healthy)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9044->9042/tcp   node_3
    6e287b35fc8e   cassandra:3.11   "docker-entrypoint.s…"   7 minutes ago   Up 7 minutes (healthy)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9043->9042/tcp   node_2

    >docker exec -it node_2 cqlsh 
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r1;
    cqlsh:music_db_r1> CONSISTENCY ONE;
    Consistency level set to ONE.
    cqlsh:music_db_r1> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable: 
    cqlsh:music_db_r1> CONSISTENCY TWO;
    Consistency level set to TWO.
    cqlsh:music_db_r1> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable: 
    cqlsh:music_db_r1> CONSISTENCY THREE;
    Consistency level set to THREE.
    cqlsh:music_db_r1> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable: 
    cqlsh:music_db_r1> CONSISTENCY QUORUM;
    Consistency level set to QUORUM.
    cqlsh:music_db_r1> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:
    cqlsh:music_db_r1> CONSISTENCY ALL;
    Consistency level set to ALL.
    cqlsh:music_db_r1> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable: 
    cqlsh:music_db_r1> CONSISTENCY SERIAL;
    Consistency level set to SERIAL.
    cqlsh:music_db_r1> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:

    >docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r2;
    cqlsh:music_db_r2> CONSISTENCY ONE;
    Consistency level set to ONE.
    cqlsh:music_db_r2> SELECT * FROM music_by_genre WHERE genre = 'Rock';

    genre | performer   | year | title            | comments
    -------+-------------+------+------------------+----------
      Rock | The Beatles | 1970 |           Dig It |     null
      Rock | The Beatles | 1969 |    Come Together |     null
      Rock | The Beatles | 1966 | Yellow Submarine |     null
      Rock | The Beatles | 1965 |          Bad Boy |     null
      Rock | The Beatles | 1965 |            Help! |        +
      Rock | The Beatles | 1963 | Please Please Me |        +

    (6 rows)
    cqlsh:music_db_r2> CONSISTENCY TWO;
    Consistency level set to TWO.
    cqlsh:music_db_r2> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:
    cqlsh:music_db_r2> CONSISTENCY THREE;
    Consistency level set to THREE.
    cqlsh:music_db_r2> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable: 
    cqlsh:music_db_r2> CONSISTENCY QUORUM;
    Consistency level set to QUORUM.
    cqlsh:music_db_r2> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:
    cqlsh:music_db_r2> CONSISTENCY ALL;
    Consistency level set to ALL.
    cqlsh:music_db_r2> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:
    cqlsh:music_db_r2> CONSISTENCY SERIAL;
    Consistency level set to SERIAL.
    cqlsh:music_db_r2> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:

    >docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> CONSISTENCY ONE;
    Consistency level set to ONE.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre WHERE genre = 'Rock';

    genre | performer | year | title                 | comments
    -------+-----------+------+-----------------------+----------
      Rock | Scorpions | 2010 |        Raised on Rock |     null
      Rock | Scorpions | 1990 |        Wind Of Change |     null
      Rock | Scorpions | 1980 |        Lady Starlight |     null
      Rock | Scorpions | 1980 |               The Zoo |     null
      Rock | Scorpions | 1979 | Another Piece of Meat |     null
      Rock | Scorpions | 1975 |             Dark Lady |        +

    (6 rows)
    cqlsh:music_db_r3> CONSISTENCY TWO;
    Consistency level set to TWO.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre WHERE genre = 'Rock';

    genre | performer | year | title                 | comments
    -------+-----------+------+-----------------------+----------
      Rock | Scorpions | 2010 |        Raised on Rock |     null
      Rock | Scorpions | 1990 |        Wind Of Change |     null
      Rock | Scorpions | 1980 |        Lady Starlight |     null
      Rock | Scorpions | 1980 |               The Zoo |     null
      Rock | Scorpions | 1979 | Another Piece of Meat |     null
      Rock | Scorpions | 1975 |             Dark Lady |        +

    (6 rows)
    cqlsh:music_db_r3> CONSISTENCY THREE;
    Consistency level set to THREE.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable: 
    cqlsh:music_db_r3> CONSISTENCY QUORUM;
    Consistency level set to QUORUM.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre WHERE genre = 'Rock';

    genre | performer | year | title                 | comments
    -------+-----------+------+-----------------------+----------
      Rock | Scorpions | 2010 |        Raised on Rock |     null
      Rock | Scorpions | 1990 |        Wind Of Change |     null
      Rock | Scorpions | 1980 |        Lady Starlight |     null
      Rock | Scorpions | 1980 |               The Zoo |     null
      Rock | Scorpions | 1979 | Another Piece of Meat |     null
      Rock | Scorpions | 1975 |             Dark Lady |        +

    (6 rows)
    cqlsh:music_db_r3> CONSISTENCY ALL;
    Consistency level set to ALL.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre WHERE genre = 'Rock';
    NoHostAvailable:
    cqlsh:music_db_r3> CONSISTENCY SERIAL;
    Consistency level set to SERIAL.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre WHERE genre = 'Rock';

    genre | performer | year | title                 | comments
    -------+-----------+------+-----------------------+----------
      Rock | Scorpions | 2010 |        Raised on Rock |     null
      Rock | Scorpions | 1990 |        Wind Of Change |     null
      Rock | Scorpions | 1980 |        Lady Starlight |     null
      Rock | Scorpions | 1980 |               The Zoo |     null
      Rock | Scorpions | 1979 | Another Piece of Meat |     null
      Rock | Scorpions | 1975 |             Dark Lady |        +

    (6 rows)

#### Task 9

Disable networking leaving all nodes switched on.

    >docker ps
    CONTAINER ID   IMAGE            COMMAND                  CREATED          STATUS                        PORTS
                 NAMES
    a94cdfddcd55   cassandra:3.11   "docker-entrypoint.s…"   21 minutes ago   Up 21 minutes (healthy)       7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9044->9042/tcp   node_3
    77a2aa5a5cde   cassandra:3.11   "docker-entrypoint.s…"   21 minutes ago   Up About a minute (healthy)   7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9042->9042/tcp   node_1
    6e287b35fc8e   cassandra:3.11   "docker-entrypoint.s…"   21 minutes ago   Up 21 minutes (healthy)       7000-7001/tcp, 7199/tcp, 9142/tcp, 9160/tcp, 0.0.0.0:9043->9042/tcp   node_2

    >docker network ls
    NETWORK ID     NAME                                DRIVER    SCOPE
    d0bef12b8c50   6_cassandra_replication_cassandra   bridge    local
    3f305e6263cd   bridge                              bridge    local
    40517e6d7a99   host                                host      local
    9ceaa2556854   none                                null      local

    >docker network disconnect d0bef12b8c50 node_1
    >docker network disconnect d0bef12b8c50 node_2
    >docker network disconnect d0bef12b8c50 node_3

#### Task 10

For Keyspace with replication factor 3 make consistency level 1. Write one record (with same primary key but different values to each node.

      >docker exec -it node_1 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 1990, 'Vogue', 'written to node_1');
      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

      genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
         Pop |                                Madonna | 1990 |                         Vogue | written to node_1
      Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
         Rock |                              Scorpions | 2010 |                Raised on Rock |              null
         Rock |                              Scorpions | 1990 |                Wind Of Change |              null
         Rock |                              Scorpions | 1980 |                Lady Starlight |              null
         Rock |                              Scorpions | 1980 |                       The Zoo |              null
         Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
         Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
      Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
      Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
         Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
         Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

      (12 rows)

      >docker exec -it node_2 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 1990, 'Vogue', 'written to node_2');
      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

      genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
         Pop |                                Madonna | 1990 |                         Vogue | written to node_2
      Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
         Rock |                              Scorpions | 2010 |                Raised on Rock |              null
         Rock |                              Scorpions | 1990 |                Wind Of Change |              null
         Rock |                              Scorpions | 1980 |                Lady Starlight |              null
         Rock |                              Scorpions | 1980 |                       The Zoo |              null
         Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
         Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
      Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
      Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
         Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
         Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

      (12 rows)

      >docker exec -it node_3 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 1990, 'Vogue', 'written to node_3');     
      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

      genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
         Pop |                                Madonna | 1990 |                         Vogue | written to node_3
      Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
         Rock |                              Scorpions | 2010 |                Raised on Rock |              null
         Rock |                              Scorpions | 1990 |                Wind Of Change |              null
         Rock |                              Scorpions | 1980 |                Lady Starlight |              null
         Rock |                              Scorpions | 1980 |                       The Zoo |              null
         Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
         Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
      Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
      Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
         Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
         Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

      (12 rows)

#### Task 11

Turn on networking and figure out which value is desided to be agreed by cluster.

    >docker network connect d0bef12b8c50 node_1 
    >docker network connect d0bef12b8c50 node_2
    >docker network connect d0bef12b8c50 node_3

    docker exec -it node_1 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> CONSISTENCY ONE;
    Consistency level set to ONE.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;
    
    genre   | performer                              | year | title                         | comments
    ---------+----------------------------------------+------+-------------------------------+-------------------
        Pop |                                Madonna | 1990 |                         Vogue | written to node_3
    Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
       Rock |                              Scorpions | 2010 |                Raised on Rock |              null
       Rock |                              Scorpions | 1990 |                Wind Of Change |              null
       Rock |                              Scorpions | 1980 |                Lady Starlight |              null
       Rock |                              Scorpions | 1980 |                       The Zoo |              null
       Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
       Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
    Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
    Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
      Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
      Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

    (12 rows)

    docker exec -it node_2 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> CONSISTENCY ONE;
    Consistency level set to ONE.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;
    
        genre   | performer                              | year | title                         | comments
    ---------+----------------------------------------+------+-------------------------------+-------------------
        Pop |                                Madonna | 1990 |                         Vogue | written to node_3
    Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
       Rock |                              Scorpions | 2010 |                Raised on Rock |              null
       Rock |                              Scorpions | 1990 |                Wind Of Change |              null
       Rock |                              Scorpions | 1980 |                Lady Starlight |              null
       Rock |                              Scorpions | 1980 |                       The Zoo |              null
       Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
       Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
    Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
    Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
      Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
      Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

    docker exec -it node_3 cqlsh
    Connected to Domin's Cluster at 127.0.0.1:9042.
    [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
    Use HELP for help.
    cqlsh> USE music_db_r3;
    cqlsh:music_db_r3> CONSISTENCY ONE;
    Consistency level set to ONE.
    cqlsh:music_db_r3> SELECT * FROM music_by_genre;
    
        genre   | performer                              | year | title                         | comments
    ---------+----------------------------------------+------+-------------------------------+-------------------
        Pop |                                Madonna | 1990 |                         Vogue | written to node_3
    Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
       Rock |                              Scorpions | 2010 |                Raised on Rock |              null
       Rock |                              Scorpions | 1990 |                Wind Of Change |              null
       Rock |                              Scorpions | 1980 |                Lady Starlight |              null
       Rock |                              Scorpions | 1980 |                       The Zoo |              null
       Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
       Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
    Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
    Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
      Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
      Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null
    

#### Task 12

Check the behavour of lightweight transactions for previous tasks in partitioned and consolidated cluster.
* https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useInsertLWT.html 


    >docker network disconnect d0bef12b8c50 node_1
    >docker network disconnect d0bef12b8c50 node_2
    >docker network disconnect d0bef12b8c50 node_3

      >docker exec -it node_1 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 2019, 'Crave', 'written to node_1') IF NOT EXISTS;
      NoHostAvailable:
      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

      genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
          Pop |                                Madonna | 1990 |                         Vogue | written to node_3
      Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
         Rock |                              Scorpions | 2010 |                Raised on Rock |              null
         Rock |                              Scorpions | 1990 |                Wind Of Change |              null
         Rock |                              Scorpions | 1980 |                Lady Starlight |              null
         Rock |                              Scorpions | 1980 |                       The Zoo |              null
         Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
         Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
      Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
      Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
        Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
        Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

      (12 rows)

      >docker exec -it node_2 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 2019, 'Crave', 'written to node_2') IF NOT EXISTS;
      NoHostAvailable:
      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

      genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
          Pop |                                Madonna | 1990 |                         Vogue | written to node_3
      Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
         Rock |                              Scorpions | 2010 |                Raised on Rock |              null
         Rock |                              Scorpions | 1990 |                Wind Of Change |              null
         Rock |                              Scorpions | 1980 |                Lady Starlight |              null
         Rock |                              Scorpions | 1980 |                       The Zoo |              null
         Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
         Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
      Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
      Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
        Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
        Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

      (12 rows)

      >docker exec -it node_3 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 2019, 'Crave', 'written to node_3') IF NOT EXISTS;
      NoHostAvailable:
      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

      genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
          Pop |                                Madonna | 1990 |                         Vogue | written to node_3
      Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
         Rock |                              Scorpions | 2010 |                Raised on Rock |              null
         Rock |                              Scorpions | 1990 |                Wind Of Change |              null
         Rock |                              Scorpions | 1980 |                Lady Starlight |              null
         Rock |                              Scorpions | 1980 |                       The Zoo |              null
         Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
         Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
      Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
      Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
        Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
        Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

      (12 rows)

    >docker network connect d0bef12b8c50 node_1 
    >docker network connect d0bef12b8c50 node_2
    >docker network connect d0bef12b8c50 node_3

      >docker exec -it node_1 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 2019, 'Crave', 'written to node_1') IF NOT EXISTS;
       
      [applied]
      -----------
            True

      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

       genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
           Pop |                                Madonna | 2019 |                         Crave | written to node_1
           Pop |                                Madonna | 1990 |                         Vogue | written to node_3
       Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
          Rock |                              Scorpions | 2010 |                Raised on Rock |              null
          Rock |                              Scorpions | 1990 |                Wind Of Change |              null
          Rock |                              Scorpions | 1980 |                Lady Starlight |              null
          Rock |                              Scorpions | 1980 |                       The Zoo |              null
          Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
          Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
       Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
       Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
         Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
         Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

       (14 rows)

      >docker exec -it node_2 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 2019, 'Crave', 'written to node_2') IF NOT EXISTS;
           
       [applied] | genre | performer | year | title | comments
      -----------+-------+-----------+------+-------+-------------------
           False |   Pop |   Madonna | 2019 | Crave | written to node_1

      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

       genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
           Pop |                                Madonna | 2019 |                         Crave | written to node_1
           Pop |                                Madonna | 1990 |                         Vogue | written to node_3
       Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
          Rock |                              Scorpions | 2010 |                Raised on Rock |              null
          Rock |                              Scorpions | 1990 |                Wind Of Change |              null
          Rock |                              Scorpions | 1980 |                Lady Starlight |              null
          Rock |                              Scorpions | 1980 |                       The Zoo |              null
          Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
          Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
       Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
       Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
         Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
         Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

       (14 rows)

      >docker exec -it node_3 cqlsh
      Connected to Domin's Cluster at 127.0.0.1:9042.
      [cqlsh 5.0.1 | Cassandra 3.11.11 | CQL spec 3.4.4 | Native protocol v4]
      Use HELP for help.
      cqlsh> USE music_db_r3;
      cqlsh:music_db_r3> CONSISTENCY ONE;
      Consistency level set to ONE.
      cqlsh:music_db_r3> INSERT INTO music_by_genre (genre, performer, year, title, comments) VALUES ('Pop', 'Madonna', 2019, 'Crave', 'written to node_3') IF NOT EXISTS;
            
       [applied] | genre | performer | year | title | comments
      -----------+-------+-----------+------+-------+-------------------
           False |   Pop |   Madonna | 2019 | Crave | written to node_1

      cqlsh:music_db_r3> SELECT * FROM music_by_genre;

       genre   | performer                              | year | title                         | comments
      ---------+----------------------------------------+------+-------------------------------+-------------------
           Pop |                                Madonna | 2019 |                         Crave | written to node_1
           Pop |                                Madonna | 1990 |                         Vogue | written to node_3
       Country |                          Johnny Duncan | 1976 |      Thinking of a Rendezvous |              null
          Rock |                              Scorpions | 2010 |                Raised on Rock |              null
          Rock |                              Scorpions | 1990 |                Wind Of Change |              null
          Rock |                              Scorpions | 1980 |                Lady Starlight |              null
          Rock |                              Scorpions | 1980 |                       The Zoo |              null
          Rock |                              Scorpions | 1979 |         Another Piece of Meat |              null
          Rock |                              Scorpions | 1975 |                     Dark Lady |                 +
       Hip-Hop | Grandmaster Flash and the Furious Five | 1982 |                   The Message |              null
       Hip-Hop |                           Public Enemy | 1989 |               Fight the Power |              null
         Blues |                           Muddy Waters | 1956 |           Got My Mojo Working |              null
         Blues |                              Otis Rush | 1958 | All Your Love (I Miss Loving) |              null

       (14 rows)